# Dengue Fever prediction

The competition that this notebook is referring to can be found at this website. 

https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/page/80/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets

from datetime import datetime

In [2]:
X = pd.read_csv('data/dengue_features_train.csv', parse_dates=['week_start_date'])
y = pd.read_csv('data/dengue_labels_train.csv')

In [3]:
X.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   city                                   1456 non-null   object        
 1   year                                   1456 non-null   int64         
 2   weekofyear                             1456 non-null   int64         
 3   week_start_date                        1456 non-null   datetime64[ns]
 4   ndvi_ne                                1262 non-null   float64       
 5   ndvi_nw                                1404 non-null   float64       
 6   ndvi_se                                1434 non-null   float64       
 7   ndvi_sw                                1434 non-null   float64       
 8   precipitation_amt_mm                   1443 non-null   float64       
 9   reanalysis_air_temp_k                  1446 non-null   float64 

We will need to impute missing data for these numeric columns.

In [5]:
X['city'].value_counts()

sj    936
iq    520
Name: city, dtype: int64

In [6]:
y.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


A plot to show the cases over time.  

In [7]:
def case_plotter(X,y):
    def plotter(city):
        city_name = {'sj': 'San Juan', 'iq': 'Iquitos'}
        plt.plot(pd.to_datetime(X[X['city'] == city]['week_start_date']), y.loc[X[X['city'] == city].index]['total_cases'])
        plt.title(f'Cases for {city_name[city]}')
        plt.ylabel('Cases')
        
    return plotter

dropdown_values = {'San Juan': 'sj', 'Iquitos': 'iq'}
widgets.interact(case_plotter(X,y), city=dropdown_values);

interactive(children=(Dropdown(description='city', options={'San Juan': 'sj', 'Iquitos': 'iq'}, value='sj'), O…

### The plan

Several things to keep in mind:

1.  We have two cities to model.  Do we want one model for both (say, with one-hot encoded values for the city) or different models (or a combined "meta-model" that wraps up two models)?
1.  There is missing data in almost every column.  Not a lot of missing data, but every numeric column seems to be missing a bit, so this will need to be imputed.  Since we have two cities, we likely want to impute the missing data separately for each city.  
1.  There is time inherent in this model, so there is likely some (mild?) seasonal/periodic behavior so likely want to engineer some features based on time.  
1.  The data for the two cities covers different time periods.  

In [8]:
categorical_features = ['city']

numeric_features = ['year', 'weekofyear', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']

time_features = ['week_start_date']

###  Imputing missing data

Let's just handle imputing the missing data separately.  Not ideal in terms of building a pipeline, but the fact that there are two cities with different means/medians for the numeric data means that either we build our own custom imputer or use two separate imputers.  

In [9]:
X[categorical_features + numeric_features].groupby('city').mean()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
city,,,,,,,,,,,,,,,,,,,,,
iq,2005.000000,26.503846,0.263869,0.238783,0.250126,0.266779,64.245736,297.869538,299.133043,295.492982,...,57.609864,88.639117,64.245736,17.096110,9.206783,27.530933,10.566197,34.004545,21.196680,62.467262
sj,1998.826923,26.503205,0.057925,0.067469,0.177655,0.165956,35.470809,299.163653,299.276920,295.109519,...,30.465419,78.568181,35.470809,16.552409,2.516267,27.006528,6.757373,31.607957,22.600645,26.785484


We build a custom imputer that will impute the missing data for the data based on a column, and returns a new DataFrame.  

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin

from sklearn.impute import SimpleImputer

In [11]:
class GroupByImputer(BaseEstimator, TransformerMixin):
    '''Assumes that the feature matrix is a pandas DataFrame.
    
       column is a column in the input feature matrix 
       We will group by column and fit a SimpleImputer for
       each group in the data, and store in a dictionary
       inside the custom transformer.  
       
       The transform method will return a new pandas DataFrame.  
    '''
    def __init__(self, column, strategy='mean'):
        self.column = column
        self.imputers = {}
        self.strategy = strategy
        
    def fit(self, X, y=None):
        grouped = X.groupby(self.column)
        cols = [c for c in X.columns if c != self.column]
        for k in grouped.groups.keys():
            imp = SimpleImputer(strategy=self.strategy)
            imp.fit(X.loc[grouped.groups[k]][cols])
            self.imputers[k] = imp
            
        return self
    
    def transform(self, X):
        result = []
        cols = [c for c in X.columns if c != self.column]
        grouped = X.groupby(self.column)
        for k in grouped.groups.keys():
            df = pd.DataFrame(self.imputers[k].transform(X.loc[grouped.groups[k]][cols]), columns=cols, index=X.loc[grouped.groups[k]].index)
            result.append(df)
            
        return pd.concat(result).sort_index()

So now create an imputer and fill in the missing data in the training set.  

In [12]:
imputer = GroupByImputer('city')
imputer.fit(X[categorical_features + numeric_features])

X[numeric_features] = imputer.transform(X[categorical_features + numeric_features])

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   city                                   1456 non-null   object        
 1   year                                   1456 non-null   float64       
 2   weekofyear                             1456 non-null   float64       
 3   week_start_date                        1456 non-null   datetime64[ns]
 4   ndvi_ne                                1456 non-null   float64       
 5   ndvi_nw                                1456 non-null   float64       
 6   ndvi_se                                1456 non-null   float64       
 7   ndvi_sw                                1456 non-null   float64       
 8   precipitation_amt_mm                   1456 non-null   float64       
 9   reanalysis_air_temp_k                  1456 non-null   float64 

###  Modeling

Now let's build some models.  This will involve engineering some features for a time series problem.  

In [14]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, \
    MaxAbsScaler, PolynomialFeatures, FunctionTransformer

from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import TimeSeriesSplit

from sklearn import metrics

In [15]:
class FourierTransformer(BaseEstimator, TransformerMixin):
    '''Tranformer that will use the weekofyear as input to compute
    sine and cosine terms for a given period.'''
    def __init__(self, period):
        self.period = period
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        return pd.DataFrame({'sin': np.sin(2*np.pi*(X-1)/self.period),
                             'cos': np.cos(2*np.pi*(X-1)/self.period)}, index=X.index)

####  Custom GroupbyEstimator

We want to handle both cities in one estimator, so we build a custom estimator that will do that for us.  

In [16]:
class GroupbyEstimator(BaseEstimator, RegressorMixin): 
    def __init__(self, column, estimator_factory):
        # column is the value to group by; estimator_factory can be
        # called to produce estimators
        self.column = column
        self.estimator_factory = estimator_factory
        self.estimators = dict()
    
    def fit(self, X, y):
        # Create an estimator and fit it with the portion in each group
        for name, df in X.groupby(self.column):
            self.estimators[name] = self.estimator_factory().fit(df, y.loc[df.index])
        return self

    def _predict(self, df):
        name = df[self.column].iloc[0]
        return self.estimators[name].predict(df)
    
    def predict(self, X):
        # Call the appropriate predict method for each row of X
        retval = pd.Series(dtype=np.float64)
        for name, indices in X.groupby(self.column).groups.items():
            retval = retval.append(pd.Series(self._predict(X.loc[indices]),
                                             indices))
        return retval.loc[X.index]

####  A function to use in FunctionTransformer

Let's engineer values from the `week_start_date` to use as a feature for a trend in the model.  We'll use a function we define and the `FunctionTransformer` to turn it into a transformer in the `Pipeline`.

In [17]:
def to_julian(series):
    return series.apply(lambda x: pd.Timestamp(x).to_julian_date()).to_frame()

In [30]:
to_julian(X['week_start_date'])

,week_start_date
0,2448011.5
1,2448018.5
2,2448025.5
3,2448032.5
4,2448039.5
...,...
1451,2455344.5
1452,2455351.5
1453,2455358.5
1454,2455365.5


####  A factory to spit out pipelines

In [35]:
def factory():
    return Pipeline([
        ('features', ColumnTransformer([
            ('fourier', FeatureUnion([
                ('yearly', FourierTransformer(52)),
                ('biannual', FourierTransformer(52//2)),
                ('semiannual', FourierTransformer(52*2))]), 'weekofyear'),
            ('numeric', 'passthrough', 
             [c for c in numeric_features if c != 'weekofyear']),
            ('julian', FunctionTransformer(to_julian), 'week_start_date')
        ])),
        ('poly', PolynomialFeatures(2, include_bias=False)),
        ('scale', StandardScaler()),
        ('regressor', 
             GridSearchCV(Ridge(), 
                          param_grid={'alpha': np.logspace(-2,2,10)},
                          cv=TimeSeriesSplit()))
    ])

#### Fitting the model

In [36]:
gbe = GroupbyEstimator('city', factory)

In [37]:
gbe.fit(X, y['total_cases'])

GroupbyEstimator(column='city',
                 estimator_factory=<function factory at 0x7f8c81d55a60>)

###  Predictions
We can get predictions, but with the additional criteria that case numbers should be non-negative.  

In [38]:
predictions = np.maximum(gbe.predict(X), 0)

In [49]:
metrics.r2_score(y['total_cases'], predictions)

0.33609340647891284

In [50]:
metrics.mean_absolute_error(y['total_cases'], predictions)

16.807059164820828

####  Plots of predictions

In [41]:
def case_plotter(X,y,predictions):
    def plotter(city):
        city_name = {'sj': 'San Juan', 'iq': 'Iquitos'}
        plt.plot(pd.to_datetime(X[X['city'] == city]['week_start_date']), y.loc[X[X['city'] == city].index]['total_cases'], label='Real cases', color='r')
        plt.plot(pd.to_datetime(X[X['city'] == city]['week_start_date']), predictions.loc[X[X['city'] == city].index], label='Predictions', color='b')
        plt.title(f'Cases for {city_name[city]}')
        plt.ylabel('Cases')
        plt.legend()
        
    return plotter

dropdown_values = {'San Juan': 'sj', 'Iquitos': 'iq'}
widgets.interact(case_plotter(X,y,pd.Series(np.maximum(gbe.predict(X),0),index=X.index)), city=dropdown_values);

interactive(children=(Dropdown(description='city', options={'San Juan': 'sj', 'Iquitos': 'iq'}, value='sj'), O…

### Submissions

In [42]:
sub_format = pd.read_csv('submissions/submission_format.csv')

In [43]:
sub_format.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,0
1,sj,2008,19,0
2,sj,2008,20,0
3,sj,2008,21,0
4,sj,2008,22,0


####  Read the test data and impute missing values for the test data

In [44]:
test = pd.read_csv('data/dengue_features_test.csv', parse_dates=['week_start_date'])

test[numeric_features] = imputer.transform(test[categorical_features + numeric_features])

####  Make and save predictions for the test data

In [45]:
submissions = sub_format.copy()
submissions['total_cases'] = np.round(np.maximum(gbe.predict(test),0)).astype(np.int64)

now = datetime.now()
filename = 'submissions/submissions-' + now.strftime('%Y-%m-%d-%H%M') + '.csv'
submissions.to_csv(filename, index=False)

In [46]:
submissions.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,10
1,sj,2008,19,0
2,sj,2008,20,0
3,sj,2008,21,19
4,sj,2008,22,7


In [47]:
submissions['total_cases'].unique()

array([10,  0, 19,  7, 24,  9, 16,  8,  2, 15, 32, 21, 34, 36, 56, 42, 44,
       68, 52, 51, 48, 50, 46, 47, 29, 11, 17,  5, 22, 18,  4,  3, 13, 25,
       35, 45, 59, 75, 40, 66, 71, 60, 39, 67, 64, 28, 57, 38, 33, 26, 14,
       23, 12, 41, 65,  1,  6, 20, 37, 54, 58, 61])

In [48]:
np.maximum(gbe.predict(test),0)

0      10.345862
1       0.000000
2       0.000000
3      19.160316
4       6.601260
         ...    
411     8.761167
412     7.629091
413     8.378471
414     5.451230
415     6.856721
Length: 416, dtype: float64